In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

In [2]:
hads2009 = pd.read_csv('hads2009RAW.csv')
hads2013 = pd.read_csv('hads2013RAW.csv')

In [4]:
hads2009.head()

,CONTROL,AGE1,ASSISTED,BEDRMS,BUILT,METRO3,NUNITS,OTHERCOST,OWNRENT,PER,...,TOTSAL,TYPE,UTILITY,VACANCY,VALUE,WEIGHT,ZADEQ,ZINC2,ZSMHC,IPOV
0,'100003130103',87,-9,3,2006,'3',1,310.000000,'1',2,...,0,1,229.166667,-6,50000,2772.262855,'1',65250,594,12956
1,'100003130203',70,-9,3,2006,'3',1,222.750000,'1',1,...,0,1,355.416667,-6,238000,2876.223614,'1',35400,684,10292
2,'100006370140',48,-9,4,1985,'5',1,79.166667,'1',4,...,58932,1,199.000000,-6,200000,2706.242657,'1',74932,1549,22159
3,'100006520140',62,-9,3,1985,'5',1,0.000000,'1',2,...,0,1,173.000000,-6,175000,2426.802292,'1',30950,817,14370
4,'100007130148',30,0,2,1980,'1',8,9.000000,'2',2,...,15600,1,113.000000,-6,-6,3089.847859,'1',15600,701,14774


In [5]:
hads2009.tail()

,CONTROL,AGE1,ASSISTED,BEDRMS,BUILT,METRO3,NUNITS,OTHERCOST,OWNRENT,PER,...,TOTSAL,TYPE,UTILITY,VACANCY,VALUE,WEIGHT,ZADEQ,ZINC2,ZSMHC,IPOV
49085,'746730070147',81,0,3,2002,'2',1,25.000000,'2',1,...,0,1,197.000000,-6,-6,2018.197497,'1',29737,1222,10292
49086,'747548160145',44,-9,4,2003,'2',1,116.666667,'1',5,...,118000,1,241.666667,-6,300000,2774.865077,'1',118000,2158,25534
49087,'747548240145',35,-9,4,2003,'2',1,10.000000,'1',3,...,99901,1,181.000000,-6,40000,2774.865077,'1',100000,439,17222
49088,'747548320145',31,0,2,2003,'1',491,25.000000,'2',1,...,40000,1,126.000000,-6,-6,1578.536304,'1',40000,1065,11177
49089,'747730090144',47,-9,5,2003,'1',1,75.000000,'1',4,...,99000,1,341.000000,-6,430000,2652.974630,'1',99000,2131,21787


In [6]:
hads2009.shape

(49090, 25)

In [7]:
hads2009.drop_duplicates()
hads2009.shape

(49090, 25)

In [8]:
hads2013.shape

(64535, 25)

In [9]:
hads2009.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49090 entries, 0 to 49089
Data columns (total 25 columns):
CONTROL          49090 non-null object
AGE1             49090 non-null int64
ASSISTED         49090 non-null int64
BEDRMS           49090 non-null int64
BUILT            49090 non-null int64
METRO3           49090 non-null object
NUNITS           49090 non-null int64
OTHERCOST        49090 non-null float64
OWNRENT          49090 non-null object
PER              49090 non-null int64
REGION           49090 non-null object
ROOMS            49090 non-null int64
STATUS           49090 non-null object
STRUCTURETYPE    49090 non-null int64
TENURE           49090 non-null object
TOTSAL           49090 non-null int64
TYPE             49090 non-null int64
UTILITY          49090 non-null float64
VACANCY          49090 non-null int64
VALUE            49090 non-null int64
WEIGHT           49090 non-null float64
ZADEQ            49090 non-null object
ZINC2            49090 non-null int64
ZSMH

In [12]:
# Creating meta data
data = []
for f in hads2009.columns:
    # Defining the role
    if f == 'VACANCY':
        role = 'target'
    elif f == 'CONTROL':
        role = 'id'
    else:
        role = 'input'
        
    # Defining the level (binary, nominal, interval or ordinal)
    if f in ['OWNRENT', 'ASSISTED']:
        level = 'binary'
    elif f in ['STRUCTURETYPE', 'ZADEQ', 'VACANCY']:
        level = 'ordinal'
    elif f in ['CONTROL', 'METRO3', 'REGION', 'STATUS', 'TENURE', 'TYPE']:
        level = 'norminal'
    elif hads2009[f].dtype == float or int:
        level = 'interval'
    
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'CONTROL':
        keep = False
        
    # Defining the data type
    dtype = hads2009[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, 
                    columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [13]:
meta

,role,level,keep,dtype
varname,,,,
CONTROL,id,norminal,False,object
AGE1,input,interval,True,int64
ASSISTED,input,binary,True,int64
BEDRMS,input,interval,True,int64
BUILT,input,interval,True,int64
METRO3,input,norminal,True,object
NUNITS,input,interval,True,int64
OTHERCOST,input,interval,True,float64
OWNRENT,input,binary,True,object
